In [2]:
import os
import os.path as op
import glob
import numpy as np
import multiprocessing
from nipype.interfaces import fsl

In [3]:
#preliminary housekeeping
home_dir = '/data/home/iballard/mag/'
subj_file = home_dir + 'subjects.txt'
subs = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
exps = ['mag']
runs = map(str,range(1,5))

In [4]:
#get cortical areas from Yeo atlas
path = home_dir + '/data/fsaverage/label/'

#get cortical areas from Michael's parcellation
cortical_areas =  glob.glob(path + '*yeo*')
cortical_areas = [x.split('/')[-1] for x in cortical_areas]
print cortical_areas

#get vmpfc roi
cortical_areas =  glob.glob(path + '*vmpfc**')
cortical_areas = [x.split('/')[-1] for x in cortical_areas]
cortical_areas = [cortical_areas[0]]
cortical_areas

['lh.vmpfc_bartre_smooth.label']

In [9]:
def make_mask(in_tuple):
    sub,roi = in_tuple
    roi_id = roi.split('.')[1]
    hemi = roi.split('.')[0]
    
    cmd = ['make_masks.py',
           '-s',
           sub,
           '-roi',
           hemi + '.' + roi_id,
            '-label',
           roi_id,
           '-sample cortex',
           '-hemi',
           hemi,
           '-debug',
           '-serial']
    cmd = ' '.join(cmd)
    os.system(cmd)

In [11]:
in_tuples = []
for sub in subs:
    for roi in cortical_areas:
        in_tuples.append((sub,roi))

In [12]:
pool = multiprocessing.Pool(50)
pool.map(make_mask,in_tuples)
pool.terminate()
pool.join()